In [1]:
from pyspark.sql import SparkSession
from operator import add
import re
import json
import time 

In [2]:
# New API
spark_session = SparkSession\
        .builder\
        .master("spark://master:7077") \
        .appName("dataset_test")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","300s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("WARN")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/16 22:11:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/16 22:11:11 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [3]:
rc_lines = spark_context.textFile('hdfs://master:9000/dataset/RC_2011-07')

rc_num = rc_lines.count()
print(rc_num)

10557466


In [4]:
rc_lines.take(5)

['{"distinguished":null,"downs":0,"created_utc":"1309478400","controversiality":0,"edited":false,"gilded":0,"author_flair_css_class":"mordekaiser","id":"c22x4aq","author":"adomorn","retrieved_on":1427302516,"score_hidden":false,"subreddit_id":"t5_2rfxx","score":1,"name":"t1_c22x4aq","author_flair_text":"[adomorn] (NA)","link_id":"t3_id1nc","archived":true,"ups":1,"parent_id":"t3_id1nc","subreddit":"leagueoflegends","body":"Good lord.  Yes."}',
 '{"distinguished":null,"created_utc":"1309478401","downs":0,"controversiality":0,"gilded":0,"edited":false,"author":"[deleted]","id":"c22x4at","author_flair_css_class":null,"retrieved_on":1427302516,"score_hidden":false,"score":-3,"subreddit_id":"t5_2qwxl","author_flair_text":null,"archived":true,"link_id":"t3_idpys","name":"t1_c22x4at","ups":-3,"parent_id":"t1_c22wz9g","body":"I don\'t know about that...","subreddit":"runescape"}',
 '{"edited":false,"gilded":0,"author":"matts2","author_flair_css_class":null,"id":"c22x4au","retrieved_on":1427302

In [5]:
rc_lines = spark_context.textFile('hdfs://master:9000/dataset/RC_2012-12')

rc_num = rc_lines.count()
print(rc_num)

26080276


In [6]:
rc_lines.take(5)

['{"edited":false,"score":8,"author_flair_text":null,"parent_id":"t1_c799b4b","author":"cassander","name":"t1_c79dblz","author_flair_css_class":null,"ups":8,"controversiality":0,"retrieved_on":1430545690,"id":"c79dblz","gilded":0,"created_utc":"1354320000","body":"&gt;It\'s the rich of today whose actions determine how big the slice of the pie is that everyone else has to share.\\n\\nthis is wrong.  wealth is not a zero sum game.","subreddit":"NeutralPolitics","distinguished":null,"score_hidden":false,"link_id":"t3_1427g3","subreddit_id":"t5_2tk0i","archived":true,"downs":0}',
 '{"downs":0,"archived":true,"distinguished":null,"score_hidden":false,"subreddit":"RandomActsOfGaming","subreddit_id":"t5_2t5mo","link_id":"t3_1416vj","body":"273","created_utc":"1354320000","controversiality":0,"retrieved_on":1430545690,"ups":1,"gilded":0,"id":"c79dbm0","author":"Whaun3","author_flair_css_class":null,"name":"t1_c79dbm0","parent_id":"t3_1416vj","author_flair_text":null,"score":1,"edited":false}'

In [7]:
def rdd_slice(rdd, start, end):
    rdd = rdd.zipWithIndex()\
            .filter(lambda kv: kv[1] >= start and kv[1] <= end) \
            .map(lambda kv: kv[0])
    return rdd     

In [8]:
rc_lines = rc_lines.cache()
rc_lines = rdd_slice(rc_lines, 0, 999999)

rc_lines.take(10)

['{"edited":false,"score":8,"author_flair_text":null,"parent_id":"t1_c799b4b","author":"cassander","name":"t1_c79dblz","author_flair_css_class":null,"ups":8,"controversiality":0,"retrieved_on":1430545690,"id":"c79dblz","gilded":0,"created_utc":"1354320000","body":"&gt;It\'s the rich of today whose actions determine how big the slice of the pie is that everyone else has to share.\\n\\nthis is wrong.  wealth is not a zero sum game.","subreddit":"NeutralPolitics","distinguished":null,"score_hidden":false,"link_id":"t3_1427g3","subreddit_id":"t5_2tk0i","archived":true,"downs":0}',
 '{"downs":0,"archived":true,"distinguished":null,"score_hidden":false,"subreddit":"RandomActsOfGaming","subreddit_id":"t5_2t5mo","link_id":"t3_1416vj","body":"273","created_utc":"1354320000","controversiality":0,"retrieved_on":1430545690,"ups":1,"gilded":0,"id":"c79dbm0","author":"Whaun3","author_flair_css_class":null,"name":"t1_c79dbm0","parent_id":"t3_1416vj","author_flair_text":null,"score":1,"edited":false}'

In [9]:
rc_lines.count()

1000000

In [10]:
spark_session.stop()